In [7]:

import pandas as pd
from sklearn.datasets import load_iris
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
import optuna
from sklearn.metrics import roc_auc_score


# Fetch iris data
iris = load_iris() 
x_data = pd.DataFrame(iris.data, columns=iris['feature_names']) 
y_data= pd.Series(data=iris.target, index=x_data.index)
# Split the data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, stratify=y_data)


WEIGHT_TUNE = True


# Define the objective function for Optuna optimization
def objective(trial, X_train, y_train, X_test, y_test):
    # Define parameters to be optimized for the weighted ensemble
    
    optuna_estimators = [
        ('lgb', LGBMClassifier(verbose=-1)),
        ('xgb', XGBClassifier()),
        ('cat', CatBoostClassifier(verbose=False))
    ]
    
    voting_classifier = VotingClassifier(
        estimators=optuna_estimators,
        voting='soft',
        weights=[
            trial.suggest_float('lgb_weight', 0.00, 10.00),
            trial.suggest_float('xgb_weight', 0.00, 10.00),
            trial.suggest_float('cat_weight', 0.00, 10.00),
        ],
    )
    
    voting_classifier.fit(x_train, y_train)
    
    predict_probs = voting_classifier.predict_proba(x_test)
    
    auc_score = roc_auc_score(y_test, predict_probs, multi_class='ovo')
    return auc_score


# Set up the sampler for Optuna optimization
weight_sampler = optuna.samplers.TPESampler(seed=42)
# Create a study object for Optuna optimization
optuna.logging.set_verbosity(optuna.logging.ERROR)
weight_study = optuna.create_study(direction="maximize", sampler=weight_sampler)
if WEIGHT_TUNE:
    # Run the optimization process
    weight_study.optimize(lambda trial: objective(trial, x_train, y_train, x_test, y_test), n_trials=50)
    # Get the best parameters after optimization
    weight_best_params = weight_study.best_params
    
    print(f'BEST WEIGHTS: {weight_best_params}')



BEST WEIGHTS: {'lgb_weight': 7.080725777960454, 'xgb_weight': 0.20584494295802447, 'cat_weight': 9.699098521619943}
